In [208]:
import numpy as np
import scipy as sp
import scipy.signal
import plotly.graph_objects as go

In [209]:
class Kuramoto_base:
    def __init__(self):
        self.times = None

    def fit(self):
        self.real_part = None
        self.Pxx_den = None
        self.signals = None
        
    def plot(self):
        fig = go.Figure(data=go.Scatter(x=self.times, y=self.real_part))
        fig.update_layout(
            title="Signal",
            xaxis_title="Time",
            yaxis_title="Signal",
            font=dict(
                family="Courier New, monospace",
                size=18)
        )
        fig.show()
        
        fig = go.Figure(data=go.Scatter(x=self.f, y=self.Pxx_den))
        fig.update_layout(
            title="PSD",
            xaxis_title="Freq (log scale)",
            #yaxis_title="Power (log scale)",
            yaxis_title="Power",
            font=dict(
                family="Courier New, monospace",
                size=18)
        )
        fig.update_xaxes(type="log")
        #fig.update_yaxes(type="log") Так все же красивее
        fig.show()
    
    def plv(self):
        signals = np.array(model.signals)
        signals /= np.abs(signals)
        conj = signals.conj()
        heatmap = np.abs(signals @ conj.T) / model.N
        fig = go.Figure(data=go.Heatmap(z=heatmap))
        fig.update_layout(
            title="PLV",
            font=dict(
                family="Courier New, monospace",
                size=18),
            autosize=False,
            width=600, 
            height=600)
        fig.show()

In [210]:
class Kuramoto(Kuramoto_base):    
    def __init__(self,w=None, K=1, N=500, sigma=1, delta_t=1e-2, max_t=1):
        super().__init__()
        self.N = N
        self.K = K
        self.delta_t = delta_t
        self.times = np.arange(0, max_t, self.delta_t)
        self.sigma = sigma
        self.w = w

    def get_w(self, t):
        if len(self.w.shape) == 1:
            return self.w
        else:
            assert self.w.shape == (len(self.times), N)
            return self.w[t, :]

    def fit(self):
        phi = np.empty((len(self.times), self.N))
        phi[0] = np.random.uniform(-np.pi, np.pi, size=self.N)
        for t in range(1, len(self.times)):
            phi[t] = self.get_w(t) * 2 * np.pi
            #sin_diffs = np.array([np.sin(phi[t - 1, i] - phi[t - 1, j]) 
            #                     for i in range(self.N) for j in range(self.N)])
            sin_diffs = np.sin(phi[t - 1, :, None] - phi[t - 1,:])
            phi[t] += self.K / self.N * sin_diffs.sum(axis=0)
            phi[t] += np.random.normal(0, self.sigma, size=self.N)
            phi[t] = phi[t] * self.delta_t + phi[t - 1] 
        
        self.signal = np.exp(1j*phi).mean(axis=1)
        self.real_part = self.signal.real
        self.f, self.Pxx_den = scipy.signal.welch(self.real_part, fs=1 / self.delta_t, nperseg=1e2)

In [211]:
N=500
w0 = np.random.uniform(10 * 0.75, 10 * 1.25, size=N)
k = Kuramoto(w=w0, K=10, N=500)

In [212]:
k.fit()

In [213]:
k.plot()

In [229]:
class Kuramoto_big(Kuramoto_base):
    def __init__(self, freqs=None, w=None, num_sim=1, K=1, sigma=1, 
                 N=500, mode='const', timesteps = [0, 15, 30, 45, 60]):
        super().__init__()
        self.delta_t = 1e-2
        self.timesteps = timesteps
        self.times = np.arange(0, timesteps[-1], self.delta_t)
        self.N = N
        self.K = K
        self.sigma = sigma
        if mode == 'const':
            self.models = [Kuramoto(w = np.random.uniform(freq * 0.75, freq * 1.25, size=N), 
                                    K=K, N=N, sigma=sigma, delta_t=self.delta_t, 
                                    max_t=timesteps[-1]) 
                           for freq in freqs]
        else:
            w = np.zeros((len(self.times), N))
            for i in range(len(timesteps) - 1):
                cur_t = np.logical_and(self.times >= timesteps[i], self.times < timesteps[i + 1])
                w[cur_t] = np.random.uniform(freqs[i] * 0.75, freqs[i] * 1.25, size=N)
                
            self.models = [Kuramoto(w=w, K=K, N=N, sigma=sigma, delta_t=self.delta_t, 
                                    max_t=timesteps[-1]) 
                           for i in range(num_sim)]

    def fit(self):
        self.Pxx_dens = [None] * len(self.models)
        self.real_parts = [None] * len(self.models)
        self.signals = [None] * len(self.models)
        for i, model in enumerate(self.models):
            model.fit()
            self.Pxx_dens[i] = model.Pxx_den
            self.real_parts[i] = model.real_part
            self.signals[i] = model.signal
        self.Pxx_dens = np.array(self.Pxx_dens)
        self.real_parts = np.array(self.real_parts)
        self.Pxx_den = self.Pxx_dens.mean(axis=0)
        self.real_part = self.real_parts.mean(axis=0)
        self.f = self.models[0].f

    def psd(self):
        real_parts = np.array(model.real_parts).reshape(self.timesteps[-1], -1)
        psd = []
        for i in range(timesteps[-1]):
            f, Pxx_den = scipy.signal.welch(real_parts[i], fs=1/10, nperseg=10)
            psd.append(f)
        fig = go.Figure(data=go.Heatmap(z=psd))
        fig.update_layout(
            title="PSD",
            font=dict(
                family="Courier New, monospace",
                size=18),
            autosize=False,
            width=600, 
            height=600)
        fig.show()


# Постоянный ток

In [230]:
model = Kuramoto_big(freqs=[10] * 10 + [15] * 5 + [25] * 5, timesteps=[1], mode='const')

In [231]:
model.fit()

In [232]:
model.plot()

In [233]:
model.plv()

# Ток с переменной частотой

In [234]:
model = Kuramoto_big(freqs=[10, 20, 25, 10], timesteps=[0, 15, 30, 45, 60], mode='non-const', num_sim=3)

In [ ]:
model.fit()

In [ ]:
model.plot()

In [ ]:
model.psd()